In [1]:
import tensorflow as tf

2023-08-24 03:21:29.230342: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 03:21:29.259639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 03:21:30.123045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__

'2.13.0'

In [3]:
import glob
# import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
# from tensorflow.python.keras import layers
from tensorflow.keras import layers, activations
import time
import nibabel as nib
from tqdm import tqdm
import gc
from scipy.ndimage import binary_erosion
from IPython import display
import datetime

import model

In [4]:
subjects = [ f.name for f in os.scandir('./Data') if f.is_dir() and f.name != '.ipynb_checkpoints']

# subjects = ['390645', '120111']

e_or_o = 0

p_size_i = 16
p_size_o = 16

print(subjects)

['390645', '545345', '106016', '120111', '101915', '351938', '705341']


In [5]:
# input_layer = tf.keras.Input(shape=(32, 32, 32, 36))
# output_layer = model.unet3d(input_layer, 36)

# generator = tf.keras.Model(input_layer, output_layer)
generator = model.gen_pix2pix()
discriminator = model.disc_pix2pix()

2023-08-24 03:21:31.900234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 03:21:31.919875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 03:21:31.919999: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
print(generator.summary())
print(discriminator.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 32, 30)]     0         []                            
                                                                                                  
 sequential (Sequential)     (None, 16, 16, 16, 64)       122880    ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 8, 8, 8, 128)         524800    ['sequential[0][0]']          
                                                                                                  
 sequential_2 (Sequential)   (None, 4, 4, 4, 256)         2098176   ['sequential_1[0][0]']        
                                                                                              

In [7]:
subjects_lr = []
subjects_hr = []
subjects_masks = []

subjects_preproc_values_lr = []
subjects_preproc_values_hr = []

subjects_t1_preproc_values = []

for subj in subjects:

    h4_imgs = []
    h4_imgs_lr = []
    dti_imgs = []
    dti_imgs_lr = []

    # DTI normalisation metrics pre-normalisation (storing max and min values in the mask) - Store for HR only
    dti_preprocs_hr = np.zeros((6,2))
    dti_preprocs_lr = np.zeros((6,2))

    print("Loading Subject {}".format(subj))
    
    for i in range(1,9):
    
        dti_data = np.array(nib.load("Data/{}/T1w/Diffusion/dt_all_{}.nii".format(subj,i)).get_fdata())
        dti_data_lr = np.array(nib.load("Data/{}/T1w/Diffusion/dt_all_lowres_2_{}.nii".format(subj,i)).get_fdata())
    
        # if i > 1:
        #     dti_data[dti_imgs[0] < 0] = 0
        #     dti_data_lr[dti_imgs[0] < 0] = 0
    
        dti_imgs.append(dti_data)
        dti_imgs_lr.append(dti_data_lr)
    
        dti_data = None
        dti_data_lr = None
    
    for i in range(1,25):
    
        h4_data = np.array(nib.load("Data/{}/T1w/Diffusion/h4_all_{}.nii".format(subj,str(i).zfill(2))).get_fdata())
        h4_data_lr = np.array(nib.load("Data/{}/T1w/Diffusion/h4_all_lowres_2_{}.nii".format(subj,str(i).zfill(2))).get_fdata())
    
        # if i > 1:
        #     h4_data[h4_data[0] < 0] = 0
        #     h4_data_lr[h4_data[0] < 0] = 0
        
        h4_imgs.append(h4_data)
        h4_imgs_lr.append(h4_data_lr)
    
        h4_data = None
        h4_data_lr = None
    
    h4_imgs = np.array(h4_imgs).transpose((1,2,3,0))
    h4_imgs_lr = np.array(h4_imgs_lr).transpose((1,2,3,0))
    
    dti_imgs = np.array(dti_imgs).transpose((1,2,3,0))
    dti_imgs_lr = np.array(dti_imgs_lr).transpose((1,2,3,0))
    
    mask = h4_imgs[...,0]

    binmask = np.zeros(mask.shape)

    binmask[mask >= 0] = 1

    mask = binary_erosion(binmask, structure=np.ones((5,5,5),np.uint8),iterations = 1)

    binmask = None

    # Calculate DTI ranges and normalise to range [0,1]
    for i in range(2,8):

        dti_preprocs_lr[i-2,:] = np.array([np.min(mask*dti_imgs_lr[...,i]), np.max(mask*dti_imgs_lr[...,i])])
        dti_imgs_lr[mask>0, i] = (dti_imgs_lr[mask>0, i] - dti_preprocs_lr[i-2,0] + 0.0001) / (dti_preprocs_lr[i-2,1] - dti_preprocs_lr[i-2,0])
        dti_imgs_lr[mask==0, i] = 0

        dti_preprocs_hr[i-2,:] = np.array([np.min(mask*dti_imgs[...,i]), np.max(mask*dti_imgs[...,i])])
        dti_imgs[mask>0, i] = (dti_imgs[mask>0, i] - dti_preprocs_hr[i-2,0] + 0.0001) / (dti_preprocs_hr[i-2,1] - dti_preprocs_hr[i-2,0])
        dti_imgs[mask==0, i] = 0


    t1w = nib.load("Data/{}/T1w/Diffusion/t1w.nii".format(subj))
    
    t1w_data = np.array(t1w.get_fdata())
    
    t1w_data_format = np.concatenate(
        (t1w_data[::2,::2,::2, None], t1w_data[::2,::2,1::2, None],
         t1w_data[::2,1::2,::2, None], t1w_data[::2,1::2,1::2, None],
         t1w_data[1::2,::2,::2, None], t1w_data[1::2,::2,1::2, None],
         t1w_data[1::2,1::2,::2, None], t1w_data[1::2,1::2,1::2, None]),
        axis=-1)

    t1w_preprocs = np.array([np.min(t1w_data_format*mask[...,None]), np.max(t1w_data_format*mask[...,None])])
    
    subjects_t1_preproc_values.append(t1w_preprocs)    
    t1w_data_format[mask>0, :] = (t1w_data_format[mask>0, :] - t1w_preprocs[0] + 0.0001) / (t1w_preprocs[1] - t1w_preprocs[0])

    subjects_preproc_values_lr.append(dti_preprocs_lr)
    subjects_preproc_values_hr.append(dti_preprocs_hr)

    lowres_input = np.concatenate((h4_imgs_lr[...,2:], t1w_data_format), axis=-1)
    hires_output = np.copy(h4_imgs[...,2:])
    
    # lowres_input = np.concatenate((h4_imgs_lr[...,2:], dti_imgs_lr[...,2:], t1w_data_format), axis=-1)
    # hires_output = np.copy(h4_imgs[...,2:])

    subjects_lr.append(lowres_input)
    subjects_hr.append(hires_output)
    subjects_masks.append(mask)


Loading Subject 390645
Loading Subject 545345
Loading Subject 106016
Loading Subject 120111
Loading Subject 101915
Loading Subject 351938
Loading Subject 705341


In [8]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

LAMBDA = 10

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l2_loss = tf.reduce_mean(tf.square(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l2_loss)

  return total_gen_loss, gan_loss, l2_loss

In [9]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=50*2000,
#     decay_rate=0.1
# )

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [10]:
checkpoint_dir = './training_checkpoints/pix2pix'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

gen_loss_metric = tf.keras.metrics.Mean('gen_loss', dtype=tf.float32)
# gen_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy('gen_accuracy')

disc_loss_metric = tf.keras.metrics.Mean('disc_loss', dtype=tf.float32)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = './logs/pix2pix_map/' + current_time + '/train'
summary_writer = tf.summary.create_file_writer(log_dir)

In [11]:
@tf.function
def train_step(input_image, target, step):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image[...,:22], target], training=True)
    disc_generated_output = discriminator([input_image[...,:22], gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//500)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//500)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//500)
    tf.summary.scalar('disc_loss', disc_loss, step=step//500)

  return gen_total_loss, disc_loss

In [12]:
def train(data, epochs):

  gen_train_mode = False

  best_cost = np.Inf

  step_count = data[0].shape[0]//4
    
  for epoch in range(epochs):

    if (epoch > 0) and ((epoch % 2) == 0):
        gen_train_mode = not gen_train_mode
      
    start = time.time()

    print("Epoch: {}".format(epoch+1))

    print("Learning rate:", generator_optimizer.learning_rate.numpy())


    mean_g_loss = 0.0
    mean_d_loss = 0.0
    total_cost = 0.0

    # [sel_indices, subjects_lr, subjects_hr, subjects_masks]

    randsels = np.random.choice(data[0].shape[0],step_count)

    lr_patches = []
    hr_patches = []

    for prog, index in enumerate(tqdm(randsels)):

        if (prog+1) % 8 != 0 or prog == (len(randsels) - 1):

            (s, i, j, k) = data[0][index]
        
            lr_patch = np.copy(data[1][s][
                                    i - p_size_i - e_or_o : i + p_size_i,
                                    j - p_size_i - e_or_o : j + p_size_i,
                                    k - p_size_i - e_or_o : k + p_size_i, :]) * \
            np.copy(data[3][s][
                                    i - p_size_i - e_or_o : i + p_size_i,
                                    j - p_size_i - e_or_o : j + p_size_i,
                                    k - p_size_i - e_or_o : k + p_size_i, None])
                               
        
            hr_patch = np.copy(data[2][s][
                                    i - p_size_o - e_or_o : i + p_size_o,
                                    j - p_size_o - e_or_o : j + p_size_o,
                                    k - p_size_o - e_or_o : k + p_size_o, :]) * \
            np.copy(data[3][s][
                                    i - p_size_o - e_or_o : i + p_size_o,
                                    j - p_size_o - e_or_o : j + p_size_o,
                                    k - p_size_o - e_or_o : k + p_size_o, None])

            lr_patches.append(lr_patch)
            hr_patches.append(hr_patch)
            
            continue
        
        lr_patches = tf.convert_to_tensor(np.array(lr_patches), dtype=tf.float32)
        hr_patches = tf.convert_to_tensor(np.array(hr_patches), dtype=tf.float32)

        g_loss, d_loss = train_step(lr_patches, hr_patches, step_count*epoch + prog)

        # if gen_train_mode:
        #     g_loss, d_loss = train_step_gen(lr_patches, hr_patches)
        # else:
        #     g_loss, d_loss = train_step_disc(lr_patches, hr_patches)

        # with summary_writer.as_default():
        #     tf.summary.scalar('gen_loss', gen_loss_metric.result(), step=epoch)
        #     tf.summary.scalar('disc_loss', disc_loss_metric.result(), step=epoch)
        
        mean_g_loss = mean_g_loss + g_loss
        mean_d_loss = mean_d_loss + d_loss

        y_preds = generator(lr_patches, training=False)

        total_cost = total_cost + tf.reduce_mean(tf.square(hr_patches - y_preds))

        lr_patches = []
        hr_patches = []
      
    # Save the model every 5 epochs
    # if (epoch + 1) % 5 == 0:
    if total_cost < best_cost:
      checkpoint.save(file_prefix = checkpoint_prefix)
      best_cost = total_cost

    # print('Generator Loss is {}'.format(float(mean_g_loss/len(randsels))))
    # print('Discriminator Loss is {}'.format(float(mean_d_loss/len(randsels))))

    print('Generator Error Metric is {}'.format(float(total_cost/(len(randsels)//4))))
      
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [13]:
(xsize, ysize, zsize, comp) = subjects_lr[0].shape

split_indices = [(i, j, k)
                 for k in np.arange(p_size_i+1,zsize-p_size_i+e_or_o,2*p_size_o+e_or_o)       
                 for j in np.arange(p_size_i+1,ysize-p_size_i+e_or_o,2*p_size_o+e_or_o)
                 for i in np.arange(p_size_i+1,xsize-p_size_i+e_or_o,2*p_size_o+e_or_o)
                ]

sel_indices = []

for s in range(len(subjects_lr)):
    for (i, j, k) in split_indices:
        ipatch_mask = subjects_masks[s][
            i - p_size_o - e_or_o : i + p_size_o,
            j - p_size_o - e_or_o : j + p_size_o,
            k - p_size_o - e_or_o : k + p_size_o]
    
        if np.max(ipatch_mask) > 0:
            sel_indices.append((s,i,j,k))

sel_indices = np.array(sel_indices)

In [ ]:
# for s in range(7):

#     print("Subject {}".format(s+1))

#     lr_patches = []
#     hr_patches = []

#     for (i, j, k) in sel_indices[np.where(sel_indices[:,0]==s),1:].squeeze():
    
    
#         lr_patch = np.copy(subjects_lr[s][
#                                 i - p_size_i - 1 : i + p_size_i,, vmax=2.0
#                                 j - p_size_i - 1 : j + p_size_i,
#                                 k - p_size_i - 1 : k + p_size_i, :])
                           
    
#         hr_patch = np.copy(subjects_hr[s][
#                                 i - p_size_o - e_or_o : i + p_size_o,
#                                 j - p_size_o - e_or_o : j + p_size_o,
#                                 k - p_size_o - e_or_o : k + p_size_o, :])
                           
#         ipatch_mask = subjects_masks[s][
#                                 i - p_size_i - e_or_o : i + p_size_i,
#                                 j - p_size_i - e_or_o : j + p_size_i,
#                                 k - p_size_i - e_or_o : k + p_size_i]

#         opatch_mask = subjects_masks[s][
#                                 i - p_size_o - e_or_o : i + p_size_o,
#                                 j - p_size_o - e_or_o : j + p_size_o,
#                                 k - p_size_o - e_or_o : k + p_size_o]

#         lr_patch[ipatch_mask == 0] = 0
#         hr_patch[opatch_mask == 0] = 0

#         lr_patches.append(lr_patch)
#         hr_patches.append(hr_patch)

#     lr_patches = np.array(lr_patches)
#     hr_patches = np.array(hr_patches)

#     # lr_patches = np.array([subjects_lr[s][
#     #                             i - p_size_i - 1 : i + p_size_i,
#     #                             j - p_size_i - 1 : j + p_size_i,
#     #                             k - p_size_i - 1 : k + p_size_i, :]
#     #                        for (i,j,k) in sel_indices[np.where(sel_indices[:,0]==s),1:].squeeze()])
    
#     # hr_patches = np.array([subjects_hr[s][
#     #                             i - p_size_o - e_or_o : i + p_size_o,
#     #                             j - p_size_o - e_or_o : j + p_size_o,
#     #                             k - p_size_o - e_or_o : k + p_size_o, :]
#     #                        for (i,j,k) in sel_indices[np.where(sel_indices[:,0]==s),1:].squeeze()])
    
# # lr_patches = np.array([subjects_lr[s][
# #                             i - p_size_i - e_or_o : i + p_size_i,
# #                             j - p_size_i - e_or_o : j + p_size_i,
# #                             k - p_size_i - e_or_o : k + p_size_i, :]
# #                        # for (i,j,k) in sel_indices[np.where(sel_indices[:,0]==s),1:].squeeze()])
# #                        for (s,i,j,k) in sel_indices])

# # hr_patches = np.array([subjects_hr[s][
# #                             i - p_size_o - e_or_o : i + p_size_o,
# #                             j - p_size_o - e_or_o : j + p_size_o,
# #                             k - p_size_o - e_or_o : k + p_size_o, :]
# #                        # for (i,j,k) in sel_indices[np.where(sel_indices[:,0]==s),1:].squeeze()])
# #                        for (s,i,j,k) in sel_indices])

#     dataset = tf.data.Dataset.from_tensor_slices((lr_patches, hr_patches))
    
train([sel_indices, subjects_lr, subjects_hr, subjects_masks], 300)

Epoch: 1
Learning rate: 0.0002


  0%|                                                   | 0/101 [00:00<?, ?it/s]2023-08-24 03:21:50.669796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-24 03:21:51.879746: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff894002610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-24 03:21:51.879771: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
2023-08-24 03:21:51.885497: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-24 03:21:52.110797: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
 32%|█████████████▎                            | 32/101 [00:08<00:11,  5.89it/s]

 40%|████████████████▋                         | 40/101 [00:08<00:08,  7.34it/s]

100%|█████████████████████████████████████████| 101/101 [00:12<00:00,  7.99it/s]


Generator Error Metric is 0.009900194592773914
Time for epoch 1 is 15.10321569442749 sec
Epoch: 2
Learning rate: 0.0002


...

100%|█████████████████████████████████████████| 101/101 [00:07<00:00, 14.25it/s]


Generator Error Metric is 0.003638234455138445
Time for epoch 285 is 7.088718414306641 sec
Epoch: 286
Learning rate: 0.0002


100%|█████████████████████████████████████████| 101/101 [00:07<00:00, 14.22it/s]


Generator Error Metric is 0.002782482421025634
Time for epoch 286 is 7.1020097732543945 sec
Epoch: 287
Learning rate: 0.0002


 48%|███████████████████▉                      | 48/101 [00:03<00:03, 13.50it/s]

In [ ]:
i = 60 
j = 60
k = 80

channel = 1

lr_patch = np.copy(subjects_lr[6][
                        i - p_size_i - e_or_o : i + p_size_i,
                        j - p_size_i - e_or_o : j + p_size_i,
                        k - p_size_i - e_or_o : k + p_size_i, :])

hr_patch = np.copy(subjects_hr[6][
                        i - p_size_o - e_or_o : i + p_size_o,
                        j - p_size_o - e_or_o : j + p_size_o,
                        k - p_size_o - e_or_o : k + p_size_o, :])

lr_mask = subjects_masks[6][
                            i - p_size_i - e_or_o : i + p_size_i,
                            j - p_size_i - e_or_o : j + p_size_i,
                            k - p_size_i - e_or_o : k + p_size_i]

lr_patch[lr_mask == 0] = 0

generated_image = generator(lr_patch[None,...], training=False)

print(discriminator([lr_patch[None,...,:22],generated_image], training=False))

print(discriminator([lr_patch[None,...,:22],hr_patch[None,...]], training=False))

plt.imshow(generated_image[0,:,:,3,channel],cmap='gray')
plt.figure()
plt.imshow(hr_patch[:,:,3,channel],cmap='gray')

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# subjects_lr
# subjects_hr
# subjects_masks

result_image = np.zeros(hires_output.shape)

for (i, j, k) in tqdm(sel_indices[np.where(sel_indices[:,0]==6),1:].squeeze()):

    lr_patch = np.copy(subjects_lr[6][
                            i - p_size_i - e_or_o : i + p_size_i,
                            j - p_size_i - e_or_o : j + p_size_i,
                            k - p_size_i - e_or_o : k + p_size_i, :])

    lr_mask = subjects_masks[6][
                                i - p_size_i - e_or_o : i + p_size_i,
                                j - p_size_i - e_or_o : j + p_size_i,
                                k - p_size_i - e_or_o : k + p_size_i]

    lr_patch[lr_mask == 0] = 0

    generated_image = generator(lr_patch[None,...], training=False)

    result_image[i - p_size_o - e_or_o : i + p_size_o,
                 j - p_size_o - e_or_o : j + p_size_o,
                 k - p_size_o - e_or_o : k + p_size_o, :] = generated_image

result_image[subjects_masks[6] == 0] = 0
    

In [ ]:
zdim = 60
channel = 0

# plt.imshow(subjects_masks[6][:,:,zdim], cmap='gray')
# plt.figure()
plt.imshow(subjects_lr[6][:,:,zdim,channel], cmap='gray')
plt.figure()
plt.imshow(subjects_hr[6][:,:,zdim,channel], cmap='gray')
plt.figure()
plt.imshow(result_image[:,:,zdim,channel], cmap='gray')
# plt.imshow((result_image[:,:,zdim,channel])*subjects_masks[6][:,:,zdim], cmap='gray')

In [ ]:
plt.imshow(subjects_lr[6][50:90,60:120,zdim,channel], cmap='gray')
plt.figure()
plt.imshow(subjects_hr[6][50:90,60:120,zdim,channel], cmap='gray')
plt.figure()
plt.imshow(result_image[50:90,60:120,zdim,channel], cmap='gray')

print(np.min(subjects_lr[6][40:80,60:120,zdim,channel]), np.min(subjects_hr[6][40:80,60:120,zdim,channel]), np.min(result_image[40:80,60:120,zdim,channel]))

In [ ]:
# generator.save_weights('Model/gen_weights')
# discriminator.save_weights('Model/disc_weights')

In [ ]:
generator.save('Model/generator{}x_dti_norm'.format(2*p_size_o + e_or_o))
discriminator.save('Model/discriminator{}x_dti_norm'.format(2*p_size_o + e_or_o))